In [1]:
import os
import anndata as ad
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import omicverse as ov
import scvi
from scvi.model.utils import mde

import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2


   ____            _     _    __                  
  / __ \____ ___  (_)___| |  / /__  _____________ 
 / / / / __ `__ \/ / ___/ | / / _ \/ ___/ ___/ _ \ 
/ /_/ / / / / / / / /__ | |/ /  __/ /  (__  )  __/ 
\____/_/ /_/ /_/_/\___/ |___/\___/_/  /____/\___/                                              

Version: 1.5.3, Tutorials: https://omicverse.readthedocs.io/


[rank: 0] Global seed set to 0


In [2]:
sc.settings.set_figure_params(dpi=100, frameon=False)
sc.set_figure_params(dpi=100)
sc.set_figure_params(figsize=(3, 3))
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (3, 3)

In [3]:
# Change the working directory to the Garfield folder (if needed)
os.chdir('/storage/liuxiaodongLab/fanxueying/mayanalysis/embryomodel_integration')
os.getcwd()

'/storage/liuxiaodongLab/fanxueying/mayanalysis/embryomodel_integration'

In [4]:
# Load dataset
os.chdir("/storage/liuxiaodongLab/fanxueying/mayanalysis/2024Aug")
adata = sc.read('human_reanno_20250108.h5ad')
adata

AnnData object with n_obs × n_vars = 33406 × 60421
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'ann_level_2', 'ann_level_3', 'ann_level_1', 'doublet', 'doublet_score', 'Unintegrated_res_0.5', 'Scanorama_res_0.5', 'LIGER_res_0.5', 'Harmony_res_0.5', 'scVI_res_0.5', 'scANVI_res_0.5', 'CCA_res_0.5', 'FastMNN_res_0.5', 'leiden_1', 'leiden_2', 'leiden_3', 'leiden_4', 'reanno', 'lineage', 'unicorns', 'combined_annotation', 'final_anno', 'final_lineage'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'CCA_res_0.5_colors', 'FastMNN_res_0.5_colors', 'Harmony_res_0.5_colors', 'LIGER_res_0.5_colors', 'Scanorama_res_0.5_colors', 'Unintegrated_res_0.5_colors', 'anno_colors', 'dendrogram_lineage', 'dendrogram_reanno', 'final_anno_colors', 'final_lineage_colors', 'harmo.anno_colors', 'hvg', 'leiden', 'leiden_3_col

In [5]:
preimplantation_adata = sc.read('/storage/liuxiaodongLab/fanxueying/mayanalysis/32_human_adata.h5ad')
preimplantation_adata

AnnData object with n_obs × n_vars = 2323 × 62754
    obs: 'day', 'ct', 'experiment', 'technology', 'n_counts', 'n_genes', 'ct_fine'
    layers: 'counts'

In [6]:
print(set(preimplantation_adata.obs["experiment"]))

{'Yan_2013', 'Meistermann_2021', 'Xiang_2020', 'Yanagida_2021', 'Xue_2013', 'Petropoulos_2016'}


In [7]:
print(set(preimplantation_adata.obs["technology"]))

{'Tang2009', 'SMARTSeq', 'SMARTSeq2'}


In [25]:
print(set(preimplantation_adata.obs["day"]))

{0.0, 1.25, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 0.75, 0.33}


In [8]:
print(set(preimplantation_adata.obs["ct_fine"]))

{'Prelineage', 'Trophectoderm_10.0', 'Trophectoderm_9.0', 'Epiblast_6.0', 'Late epiblast', '8C_3.0', 'Primitive Endoderm', 'Morula_4.0', 'Trophectoderm_6.0', 'Trophectoderm_8.0', 'Epiblast_7.0', 'Trophectoderm_7.0', 'Unknown', 'Inner Cell Mass', 'Trophectoderm_5.0'}


In [9]:
import re
# Define a function to clean up cell type annotations
def clean_cell_type(annotation):
    # Remove everything after the first underscore (_), including the underscore itself
    cleaned = re.sub(r'_.*', '', annotation)
    # Strip any leading/trailing whitespace
    cleaned = cleaned.strip()
    return cleaned

# Apply the cleanup function to the existing cell type annotations
cleaned_annotations = preimplantation_adata.obs["ct_fine"].apply(clean_cell_type)

print(set(cleaned_annotations))


{'8C', 'Trophectoderm', 'Late epiblast', 'Primitive Endoderm', 'Morula', 'Epiblast', 'Prelineage', 'Unknown', 'Inner Cell Mass'}


In [10]:
preimplantation_adata.obs["final_lineage"] = cleaned_annotations
# Verify the new column
print("New 'lineage' column added to embryomodel.obs:")
print(preimplantation_adata.obs["final_lineage"].value_counts())

New 'lineage' column added to embryomodel.obs:
Trophectoderm         1270
Unknown                609
Inner Cell Mass        140
Epiblast               122
Primitive Endoderm      59
Prelineage              39
Late epiblast           35
8C                      30
Morula                  19
Name: final_lineage, dtype: int64


In [11]:
# Subset the AnnData object to exclude cells with "final_lineage" == "Unknown"
filtered_preimplantation_adata = preimplantation_adata[preimplantation_adata.obs["final_lineage"] != "Unknown"].copy()

# Verify the filtered dataset
print(f"Filtered dataset shape: {filtered_preimplantation_adata.shape}")
print("Unique values in 'final_lineage':")
print(filtered_preimplantation_adata.obs["final_lineage"].unique())

Filtered dataset shape: (1714, 62754)
Unique values in 'final_lineage':
['Trophectoderm' 'Inner Cell Mass' 'Primitive Endoderm' 'Epiblast'
 'Late epiblast' 'Prelineage' '8C' 'Morula']


In [12]:
# Rename the columns in preimplantation_adata.obs
filtered_preimplantation_adata.obs.rename(
    columns={
        "day": "stage",
        "n_counts": "nCount_RNA",
        "n_genes": "nFeature_RNA",
        "ct": "ann_level_2",
        "ct_fine": "ann_level_3",
        "technology": "platform",
        "experiment": "orig.ident"
    },
    inplace=True
)

# Verify the renamed columns
print("Renamed columns in filtered_preimplantation_adata.obs:")
print(filtered_preimplantation_adata.obs.columns)

# Optionally inspect the first few rows of the updated .obs DataFrame
print("\nFirst few rows of updated .obs:")
print(filtered_preimplantation_adata.obs.head())

Renamed columns in filtered_preimplantation_adata.obs:
Index(['stage', 'ann_level_2', 'orig.ident', 'platform', 'nCount_RNA',
       'nFeature_RNA', 'ann_level_3', 'final_lineage'],
      dtype='object')

First few rows of updated .obs:
                       stage      ann_level_2        orig.ident   platform  \
index                                                                        
ERX1121097_ERX1121097    5.0    Trophectoderm  Petropoulos_2016  SMARTSeq2   
ERX1121101_ERX1121101    5.0    Trophectoderm  Petropoulos_2016  SMARTSeq2   
ERX1121098_ERX1121098    5.0    Trophectoderm  Petropoulos_2016  SMARTSeq2   
ERX1121099_ERX1121099    5.0  Inner Cell Mass  Petropoulos_2016  SMARTSeq2   
ERX1121100_ERX1121100    5.0    Trophectoderm  Petropoulos_2016  SMARTSeq2   

                       nCount_RNA  nFeature_RNA        ann_level_3  \
index                                                                
ERX1121097_ERX1121097   5571678.0         12456  Trophectoderm_5.0   
ERX112

In [13]:
print(adata.var.index[:5])
print(filtered_preimplantation_adata.var.index[:20])

Index(['MIR1302-2HG', 'OR4F5', 'AL627309.1', 'AL627309.3', 'AL627309.5'], dtype='object')
Index(['ENSG00000279928', 'ENSG00000228037', 'ENSG00000142611',
       'ENSG00000284616', 'ENSG00000157911', 'ENSG00000269896',
       'ENSG00000228463', 'ENSG00000260972', 'ENSG00000224340',
       'ENSG00000226374', 'ENSG00000229280', 'ENSG00000142655',
       'ENSG00000232596', 'ENSG00000235054', 'ENSG00000231510',
       'ENSG00000149527', 'ENSG00000284739', 'ENSG00000171621',
       'ENSG00000272235', 'ENSG00000284694'],
      dtype='object')


In [14]:
import pandas as pd

# Load the GTF file
gtf_file = "/storage/liuxiaodongLab/fanxueying/gencode.v47lift37.annotation.gtf"
gtf = pd.read_csv(gtf_file, sep="\t", comment="#", header=None)

# Extract gene_id and gene_name
gene_annotations = gtf[gtf[2] == "gene"]
gene_annotations = gene_annotations[8].str.extract(r'gene_id "(.*?)";.*gene_name "(.*?)";')

# Rename columns
gene_annotations.columns = ["gene_id", "gene_symbol"]

# Save the mapping file
gene_annotations.to_csv("gene_id_to_symbol_mapping.tsv", sep="\t", index=False)



In [15]:
# Load the gene ID-to-symbol mapping file
gene_mapping = pd.read_csv("gene_id_to_symbol_mapping.tsv", sep="\t")

# Verify the mapping
print(gene_mapping.head())

# Remove version numbers and suffixes from the gene IDs in the mapping file
gene_mapping['gene_id'] = gene_mapping['gene_id'].str.split(".").str[0]  # Remove version numbers
gene_mapping['gene_id'] = gene_mapping['gene_id'].str.split("_").str[0]  # Remove suffixes

# Verify the updated gene IDs in the mapping file
print("First few rows of the cleaned mapping file:")
print(gene_mapping.head())

               gene_id gene_symbol
0  ENSG00000308415.1_1     DDX11L2
1  ENSG00000290825.2_2    DDX11L16
2  ENSG00000223972.6_6     DDX11L1
3  ENSG00000310526.1_1      WASH7P
4  ENSG00000227232.6_7      WASH7P
First few rows of the cleaned mapping file:
           gene_id gene_symbol
0  ENSG00000308415     DDX11L2
1  ENSG00000290825    DDX11L16
2  ENSG00000223972     DDX11L1
3  ENSG00000310526      WASH7P
4  ENSG00000227232      WASH7P


In [16]:
# Step 3: Inspect and preprocess .var.index in filtered_preimplantation_adata
print("Data type of filtered_preimplantation_adata.var.index before conversion:")
print(filtered_preimplantation_adata.var.index.dtype)

# Convert .var.index to string type
filtered_preimplantation_adata.var.index = filtered_preimplantation_adata.var.index.astype(str)

# Verify the updated data type
print("Data type of filtered_preimplantation_adata.var.index after conversion:")
print(filtered_preimplantation_adata.var.index.dtype)

# Remove version numbers and suffixes from the gene IDs in .var.index
filtered_preimplantation_adata.var.index = filtered_preimplantation_adata.var.index.str.split(".").str[0]
filtered_preimplantation_adata.var.index = filtered_preimplantation_adata.var.index.str.split("_").str[0]

# Verify the updated .var.index
print("Updated gene IDs in filtered_preimplantation_adata.var.index:")
print(filtered_preimplantation_adata.var.index)

# Step 4: Map gene IDs to symbols
gene_id_to_symbol = dict(zip(gene_mapping['gene_id'], gene_mapping['gene_symbol']))
filtered_preimplantation_adata.var['gene_symbol'] = filtered_preimplantation_adata.var.index.map(gene_id_to_symbol)
filtered_preimplantation_adata = filtered_preimplantation_adata[:, filtered_preimplantation_adata.var['gene_symbol'].notna()].copy()
filtered_preimplantation_adata.var.index = filtered_preimplantation_adata.var['gene_symbol']

# Verify the updated .var index
print("Updated .var index for filtered_preimplantation_adata:")
print(filtered_preimplantation_adata.var.index)


Data type of filtered_preimplantation_adata.var.index before conversion:
object
Data type of filtered_preimplantation_adata.var.index after conversion:
object
Updated gene IDs in filtered_preimplantation_adata.var.index:
Index(['ENSG00000279928', 'ENSG00000228037', 'ENSG00000142611',
       'ENSG00000284616', 'ENSG00000157911', 'ENSG00000269896',
       'ENSG00000228463', 'ENSG00000260972', 'ENSG00000224340',
       'ENSG00000226374',
       ...
       'ENSG00000276351', 'ENSG00000276345', 'ENSG00000273532',
       'ENSG00000275063', 'ENSG00000277856', 'ENSG00000271254',
       'ENSG00000275987', 'ENSG00000268674', 'ENSG00000277475',
       'ENSG00000275405'],
      dtype='object', length=62754)
Updated .var index for filtered_preimplantation_adata:
Index(['ENSG00000228037', 'PRDM16', 'ENSG00000284616', 'PEX10',
       'ENSG00000269896', 'ENSG00000228463', 'ENSG00000260972', 'RPL21P21',
       'LINC01345', 'EEF1DP6',
       ...
       'MT-TH', 'MT-TS2', 'MT-TL2', 'MT-ND5', 'MT-ND6', 'M

In [17]:
# Step 4: Check for duplicate gene IDs
duplicates = filtered_preimplantation_adata.var.index[filtered_preimplantation_adata.var.index.duplicated()]
print(f"Number of duplicate gene IDs: {len(duplicates)}")
if len(duplicates) > 0:
    print("Sample of duplicate gene IDs:")
    print(duplicates[:10])

# Drop duplicate gene IDs, keeping only the first occurrence
filtered_preimplantation_adata = filtered_preimplantation_adata[:, ~filtered_preimplantation_adata.var.index.duplicated()].copy()

# Verify the updated .var.index
print("Updated .var.index after removing duplicates:")
print(filtered_preimplantation_adata.var.index)

Number of duplicate gene IDs: 1537
Sample of duplicate gene IDs:
Index(['Y_RNA', 'Y_RNA', 'Y_RNA', 'Y_RNA', 'Y_RNA', 'Y_RNA', 'snoU13', 'Y_RNA',
       'U3', 'Y_RNA'],
      dtype='object', name='gene_symbol')
Updated .var.index after removing duplicates:
Index(['ENSG00000228037', 'PRDM16', 'ENSG00000284616', 'PEX10',
       'ENSG00000269896', 'ENSG00000228463', 'ENSG00000260972', 'RPL21P21',
       'LINC01345', 'EEF1DP6',
       ...
       'MT-TH', 'MT-TS2', 'MT-TL2', 'MT-ND5', 'MT-ND6', 'MT-TE', 'MT-CYB',
       'MT-TT', 'MT-TP', 'AL355149.2'],
      dtype='object', name='gene_symbol', length=58122)


In [18]:
# Step 5: Map gene IDs to symbols
gene_id_to_symbol = dict(zip(gene_mapping['gene_id'], gene_mapping['gene_symbol']))

# Add a 'gene_symbol' column to .var for both datasets
filtered_preimplantation_adata.var['gene_symbol'] = filtered_preimplantation_adata.var.index.map(gene_id_to_symbol)
adata.var['gene_symbol'] = adata.var.index.map(gene_id_to_symbol)

# Verify the updated .var with gene symbols
print("Filtered preimplantation .var with gene symbols:")
print(filtered_preimplantation_adata.var.head())
print("Adata .var with gene symbols:")
print(adata.var.head())

# Step 6: Find common genes based on gene IDs
common_genes = filtered_preimplantation_adata.var.index.intersection(adata.var.index)
print(f"Number of common genes: {len(common_genes)}")
if len(common_genes) == 0:
    print("No common genes found! Check the gene identifiers in both datasets.")
else:
    print("Sample of common genes:")
    print(list(common_genes)[:10])

# Step 7: Subset to common genes
adata_common = adata[:, common_genes].copy()
filtered_preimplantation_adata_common = filtered_preimplantation_adata[:, common_genes].copy()

# Verify the shapes of the subsetted datasets
print(f"Subsetted adata shape: {adata_common.shape}")
print(f"Subsetted filtered_preimplantation_adata shape: {filtered_preimplantation_adata_common.shape}")


Filtered preimplantation .var with gene symbols:
                     gene_symbol
gene_symbol                     
ENSG00000228037  ENSG00000228037
PRDM16                       NaN
ENSG00000284616  ENSG00000284616
PEX10                        NaN
ENSG00000269896  ENSG00000269896
Adata .var with gene symbols:
                features  n_cells  highly_variable     means  dispersions  \
MIR1302-2HG  MIR1302-2HG       20            False  0.000090     0.118350   
OR4F5              OR4F5        5            False  0.000149     0.300203   
AL627309.1    AL627309.1      220            False  0.000821     0.203147   
AL627309.3    AL627309.3        4            False  0.000004     0.015082   
AL627309.5    AL627309.5      555            False  0.002144     0.213056   

             dispersions_norm  highly_variable_nbatches  \
MIR1302-2HG          0.564889                         1   
OR4F5                1.220983                         1   
AL627309.1          -0.084739                     

In [19]:
# Define the list of selected .obs columns to retain
selected_obs_columns = [
    'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt',
    'species', 'embryo', 'platform', 'ann_level_2', 'ann_level_3', 'ann_level_1',
    'reanno', 'lineage', 'unicorns', 'combined_annotation', 'final_anno', 'final_lineage'
]

# Step 1: Add missing .obs columns to filtered_preimplantation_adata_common
# Reindex to include all selected columns, filling missing ones with NaN
filtered_preimplantation_adata_common.obs = filtered_preimplantation_adata_common.obs.reindex(columns=selected_obs_columns)

# Fill missing values with NaN
filtered_preimplantation_adata_common.obs = filtered_preimplantation_adata_common.obs.fillna(np.nan)

# Verify the updated .obs columns in filtered_preimplantation_adata_common
print("Updated .obs columns in filtered_preimplantation_adata_common:")
print(filtered_preimplantation_adata_common.obs.columns)

# Step 2: Merge datasets using inner join
merged_adata = sc.concat(
    [adata_common, filtered_preimplantation_adata_common],
    axis=0,  # Concatenate along the cell axis
    join="inner"  # Use inner join to retain only common columns
)

# Add a new column in .obs to distinguish the source of each cell
merged_adata.obs["dataset"] = ["adata"] * adata_common.n_obs + ["preimplantation"] * filtered_preimplantation_adata_common.n_obs

# Verify the merged dataset
print(f"Merged dataset shape: {merged_adata.shape}")
print("First few rows of merged .obs:")
print(merged_adata.obs.head())

Updated .obs columns in filtered_preimplantation_adata_common:
Index(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage',
       'percent.mt', 'species', 'embryo', 'platform', 'ann_level_2',
       'ann_level_3', 'ann_level_1', 'reanno', 'lineage', 'unicorns',
       'combined_annotation', 'final_anno', 'final_lineage'],
      dtype='object')
Merged dataset shape: (35120, 45796)
First few rows of merged .obs:
                               orig.ident  nCount_RNA  nFeature_RNA   sample  \
AAACCTGAGACGCTTT-1_1_1_1_1_1_1       mole    112028.0          9381  Sample1   
AACGTTGAGACGCAAC-1_1_1_1_1_1_1       mole    191885.0          8618  Sample1   
AACTCAGCAGACGCCT-1_1_1_1_1_1_1       mole    189565.0          7942  Sample1   
AAGACCTTCGGCGCTA-1_1_1_1_1_1_1       mole     25545.0          4869  Sample1   
AAGCCGCCATATGGTC-1_1_1_1_1_1_1       mole    107601.0         10074  Sample1   

                                 stage  percent.mt       species  \
AAACCTGAGACGCTTT-1_1_1_1_1_

In [20]:
merged_adata

AnnData object with n_obs × n_vars = 35120 × 45796
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'ann_level_2', 'ann_level_3', 'ann_level_1', 'reanno', 'lineage', 'unicorns', 'combined_annotation', 'final_anno', 'final_lineage', 'dataset'
    layers: 'counts'

In [21]:
# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print(f"Current Working Directory: {current_directory}")

Current Working Directory: /storage/liuxiaodongLab/fanxueying/mayanalysis/2024Aug


In [22]:
# Inspect the .obs DataFrame
print("Columns in merged_adata.obs:")
print(merged_adata.obs.columns)

# Check the data types of each column
print("\nData types of merged_adata.obs:")
print(merged_adata.obs.dtypes)

# Inspect the first few rows of merged_adata.obs
print("\nFirst few rows of merged_adata.obs:")
print(merged_adata.obs.head())

# Identify non-string columns
non_string_columns = [col for col, dtype in merged_adata.obs.dtypes.items() if dtype != "object"]
print(f"Non-string columns: {non_string_columns}")

# Convert non-string columns to strings
for col in non_string_columns:
    merged_adata.obs[col] = merged_adata.obs[col].astype(str)

# Verify the updated data types
print("\nUpdated data types of merged_adata.obs:")
print(merged_adata.obs.dtypes)

# Handle missing values
merged_adata.obs = merged_adata.obs.fillna("NA")

# Verify the updated .obs DataFrame
print("\nFirst few rows of merged_adata.obs after handling missing values:")
print(merged_adata.obs.head())


Columns in merged_adata.obs:
Index(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage',
       'percent.mt', 'species', 'embryo', 'platform', 'ann_level_2',
       'ann_level_3', 'ann_level_1', 'reanno', 'lineage', 'unicorns',
       'combined_annotation', 'final_anno', 'final_lineage', 'dataset'],
      dtype='object')

Data types of merged_adata.obs:
orig.ident             category
nCount_RNA              float64
nFeature_RNA              int64
sample                   object
stage                    object
percent.mt              float64
species                  object
embryo                   object
platform               category
ann_level_2            category
ann_level_3            category
ann_level_1              object
reanno                   object
lineage                  object
unicorns                 object
combined_annotation      object
final_anno               object
final_lineage            object
dataset                  object
dtype: object

First few r

In [26]:
print(set(merged_adata.obs["stage"]))

{0.0, 1.25, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 'E9_IVC', 10.0, 9.0, 0.75, 'E6', 'CS10', 'CS7', 'E11_IVC', 'E7_IVC', 'E10_IVC', 'E8_IVC', 'E12_IVC', 'E6_IVC', 'E14_IVC', 0.33, 'E13_IVC'}


In [27]:
# Convert the 'stage' column to strings
merged_adata.obs['stage'] = merged_adata.obs['stage'].astype(str)

# Verify the updated 'stage' column
print("\nUpdated unique values in merged_adata.obs['stage']:")
print(merged_adata.obs['stage'].unique())


Updated unique values in merged_adata.obs['stage']:
['E9_IVC' 'E11_IVC' 'CS10' 'E10_IVC' 'E12_IVC' 'E13_IVC' 'E14_IVC' 'CS7'
 'E6_IVC' 'E7_IVC' 'E8_IVC' 'E6' '5.0' '6.0' '7.0' '10.0' '8.0' '9.0'
 '2.0' '3.0' '1.25' '4.0' '0.0' '0.75' '0.33']


In [28]:
merged_adata.raw.var.rename(columns={'_index': 'index'}, inplace=True)
merged_adata.write_h5ad(filename="embryo_pre_post_20250310.h5ad")